In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('deliveries.csv')
venue = pd.read_csv('venue.csv')
players= pd.read_csv('players_info_with_keys.csv')
match = pd.read_csv('match_details.csv')


In [3]:
data.drop_duplicates(inplace=True)

In [4]:
data[['extras','wides','noballs','byes','legbyes','penalty']]= data[['extras','wides','noballs','byes','legbyes','penalty']].fillna(0).astype('int')

In [5]:
data['season'].value_counts()

2013       18177
2022       17912
2012       17767
2011       17013
2020/21    14559
2009/10    14498
2021       14425
2019       14312
2014       14300
2018       14286
2016       14096
2017       13862
2015       13652
2009       13606
2007/08    13489
Name: season, dtype: int64

In [6]:
#most runs
batsman = data.groupby('striker').sum().sort_values('runs_off_bat',ascending= False)
batsman_run= batsman['runs_off_bat'].reset_index()
batsman_run


,striker,runs_off_bat
0,V Kohli,6634
1,S Dhawan,6244
2,DA Warner,5883
3,RG Sharma,5881
4,SK Raina,5536
...,...,...
600,C Nanda,0
601,Akash Deep,0
602,S Ladda,0
603,V Pratap Singh,0


In [7]:
data_selected = batsman_run.head(10)
data_selected = data_selected.rename(columns={'striker':'Batsman','runs_off_bat':'y'})
data_selected

,Batsman,y
0,V Kohli,6634
1,S Dhawan,6244
2,DA Warner,5883
3,RG Sharma,5881
4,SK Raina,5536
5,AB de Villiers,5181
6,CH Gayle,4997
7,MS Dhoni,4978
8,RV Uthappa,4954
9,KD Karthik,4377


In [16]:
#total balls played 
#df_total_ball_faced = df[df['extras_type']!='wides'].groupby('batsman').count().sort_values(by='batsman_runs', ascending=False)['batsman_runs']
ball_faced = data[data['wides']==0].groupby('striker').count().sort_values('runs_off_bat',ascending=False)['runs_off_bat']
ball_faced
player = batsman_run
player=player.merge(ball_faced,left_on='striker',right_on='striker')
player = player[player['runs_off_bat_y']>200]
player=player.reset_index()
player['strikerate'] = (player['runs_off_bat_x']/player['runs_off_bat_y'])*100
player=player.sort_values('strikerate',ascending=False)
player=player.reset_index()
player=player.rename(columns={'runs_off_bat_x':'runs','runs_off_bat_y':'balls'})
player=player.drop(columns=['runs','balls','level_0','index'])
player

,striker,strikerate
0,DA Warner,1810.153846
1,CH Gayle,1759.507042
2,S Dhawan,1669.518717
3,SR Watson,1583.673469
4,RV Uthappa,1519.631902
...,...,...
65,PP Chawla,168.299712
66,A Mishra,155.364807
67,R Vinay Kumar,137.168142
68,P Kumar,130.268199


In [9]:
do = data[data['over']>15].groupby('striker').sum().sort_values('runs_off_bat',ascending= False)
do = do['runs_off_bat'].reset_index()

In [10]:
data= data[(data['over']>15)]
ball_faced = data[(data['wides']==0) ].groupby('striker').count().sort_values('runs_off_bat',ascending=False)['runs_off_bat']
player = do
player=player.merge(ball_faced,left_on='striker',right_on='striker')
player = player[player['runs_off_bat_y']>200]
player=player.reset_index()
player['strikerate'] = (player['runs_off_bat_x']/player['runs_off_bat_y'])*100
player=player.sort_values('strikerate',ascending=False)
player=player.reset_index()
player=player.rename(columns={'runs_off_bat_x':'runs','runs_off_bat_y':'balls'})
player=player.drop(columns=['runs','balls','level_0','index'])
player

,striker,strikerate
0,AB de Villiers,222.911695
1,CH Gayle,204.577465
2,RR Pant,200.000000
3,JC Buttler,197.916667
4,AD Russell,197.196262
...,...,...
65,PP Chawla,122.190202
66,R Vinay Kumar,115.929204
67,P Kumar,112.643678
68,A Mishra,106.008584


In [11]:
data['over'].info()

<class 'pandas.core.series.Series'>
Int64Index: 51311 entries, 37 to 225935
Series name: over
Non-Null Count  Dtype
--------------  -----
51311 non-null  int64
dtypes: int64(1)
memory usage: 801.7 KB


In [12]:
#best batting average
temp = data
temp['count'] = 1
temp=temp.groupby("player_dismissed").sum().reset_index()
temp=temp.sort_values('count',ascending=False)
temp= temp[['player_dismissed','count']]
temp

,player_dismissed,count
276,MS Dhoni,85
203,KA Pollard,80
208,KD Karthik,64
345,RA Jadeja,64
350,RG Sharma,60
...,...,...
54,Akash Deep,1
342,R Shukla,1
112,DE Bollinger,1
45,AS Joseph,1


In [13]:
#batting Average
temp = data
temp['count'] = 1
temp=temp.groupby("player_dismissed").sum().reset_index()
temp=temp.sort_values('count',ascending=False)
temp= temp[['player_dismissed','count']]
player = batsman_run
player= player.merge(temp,left_on='striker',right_on='player_dismissed')
player = player[player['runs_off_bat']>500]
player['bat_avg'] = (player['runs_off_bat']/player['count'])
top_avg= player.sort_values('bat_avg',ascending=False).reset_index()
top_avg=top_avg[['striker','bat_avg']]


In [14]:
player=player.drop('player_dismissed',axis=1)
player=player.sort_values('run',ascending= False)
player

KeyError: 'run'

In [ ]:
data = data.drop(columns = ['striker_id','non_striker_id','bowler_id'])

In [ ]:
data.sample(2)

In [ ]:
#total runs scored so far
data['runs']= data['runs_off_bat'].fillna(0) +data['extras'].fillna(0) +data['wides'].fillna(0) +data['noballs'].fillna(0)+data['byes'].fillna(0)+data['legbyes'].fillna(0)+data['penalty'].fillna(0)
data.sample(2)

In [ ]:
total_runs= data['runs'].sum()
total_runs

In [ ]:
#most wickets
data['wicket_type'].value_counts()

In [ ]:
l= ['caught','bowled','lbw','stumped','caught and bowled']
data_bowler = data[data['wicket_type'].isin(l)]
data_bowler['wicket']= 1


In [ ]:
bowler = data_bowler.groupby('bowler').sum()
bowler=bowler['wicket'].sort_values(ascending=False).reset_index().head()

In [ ]:
bowler

In [ ]:
wickets= data['wicket_type'].value_counts().reset_index()
wickets

In [ ]:
total_wickets= wickets['wicket_type'].sum()
total_wickets

In [ ]:
#most fours
fours_df= data[data['runs_off_bat']==4]
total_fours= fours_df
total_fours

In [ ]:

most_fours_batsman= fours_df.groupby('striker')
most_fours_batsman=most_fours_batsman['runs_off_bat'].sum().sort_values(ascending=False).reset_index()
most_fours_batsman['runs_off_bat'] = (most_fours_batsman['runs_off_bat']/4).astype('int')
most_fours_batsman.head()

In [ ]:
player=player.merge(most_fours_batsman,left_on='striker',right_on='striker')
player=player.rename(columns={'count':'out','runs_off_bat':"4s"})
player

In [ ]:
#Most six

#most fours
six_df= data[data['runs_off_bat']==6]
total_six= six_df.shape
print(total_six)
most_six_batsman= six_df.groupby('striker')
most_six_batsman=most_six_batsman['runs_off_bat'].sum().sort_values(ascending=False).reset_index()
most_six_batsman['runs_off_bat'] = (most_six_batsman['runs_off_bat']/6).astype('int')
most_six_batsman.head()

In [ ]:
player=player.merge(most_six_batsman,left_on='striker',right_on='striker')
player=player.rename(columns={'count':'out','runs_off_bat':"6s"})
player

In [ ]:
#most dots
dots_df= data[data['runs_off_bat']==0]
dots_df['striker'].value_counts().reset_index().head()


In [ ]:
#most ducks



In [ ]:
#best strike rates 


In [ ]:
#total players
players.drop_duplicates()

In [ ]:
players.shape[0]

In [ ]:
players['Playing Position'].value_counts()

In [ ]:
#total batsman
pp=['Top order Batter','Middle order Batter','Opening Batter']
players[players['Playing Position'].isin(pp)]

In [ ]:
#total bowlers
players[players['Playing Position']=='Bowler']

In [ ]:
#total allrounders
pa=['Allrounder','Bowling Allrounder','Batting Allrounder']
players[players['Playing Position'].isin(pa)]

In [ ]:
#total wicketkeepr
pw = ['Wicketkeeper','Wicketkeeper Batter']
players[players['Playing Position'].isin(pw)].shape

In [ ]:
match['season'].value_counts().shape[0]

In [ ]:
#total matches
match.drop_duplicates().shape

In [ ]:
#total venues
venue.drop_duplicates().shape

In [ ]:
#most frequent venue
most_venue = match.groupby('venueID')['match_id'].count().reset_index()
most_venue.head()

In [ ]:
venue = venue.merge(most_venue,right_on='venueID',left_on='id')

In [ ]:
most_used_stadium = venue.sort_values('match_id',ascending=False)
most_used_stadium = most_used_stadium[['venue','match_id']]
most_used_stadium.rename(columns={'match_id':'Matches Played'})
most_used_stadium.head()

In [ ]:
#most MOM
match['player_of_match'].value_counts().reset_index().head()

In [ ]:
#most wins by a team
match['winner'].value_counts().reset_index().head()

In [ ]:
#biggest wins by runs
biggest_winruns = match.sort_values('win_by',ascending= False)
biggest_winruns = biggest_winruns[['team1','team2','season','date','toss_winner','winner','win_by','player_of_match','venueID']].head(10)
biggest_winruns=biggest_winruns.merge(venue,right_on='id',left_on='venueID')
biggest_winruns = biggest_winruns[['team1','team2','season','date','toss_winner','winner','win_by','player_of_match','venue','city']].sort_values('win_by',ascending= False)
biggest_winruns

In [ ]:
data['player_dismissed'] = data['player_dismissed'].fillna('')
data.info()


In [ ]:
#fifties
new_df= data.groupby(['match_id','striker'])['runs_off_bat'].sum().reset_index()
new_df
fifties = new_df[(new_df['runs_off_bat']>=50) & (new_df['runs_off_bat']<100)]
total_fifty= fifties.shape[0]
print(total_fifty)
fifties.drop(columns = ['match_id'],inplace=True)
fifties= fifties.groupby('striker').count().sort_values('runs_off_bat',ascending=False)
fifties = fifties.reset_index()
fifties

In [ ]:
#100s
hundreds = new_df[(new_df['runs_off_bat']>=100)]
total_hundreds= hundreds.shape[0]
print(total_hundreds)
hundreds.drop(columns = ['match_id'],inplace=True)
hundreds= hundreds.groupby('striker').count().reset_index()
hundreds.sort_values('runs_off_bat',ascending=False).head()

In [ ]:
#highest scores

new_df=new_df.sort_values('runs_off_bat',ascending=False)
new_df=new_df.reset_index()
new_df = new_df.drop(columns = ['match_id','index'])
new_df

In [ ]:
#most ducks
new_df= new_df[new_df['runs_off_bat']==0]
ducks_df= new_df.groupby('striker').count()
ducks_df.sort_values('runs_off_bat',ascending=False).reset_index()

In [ ]:
data.head()

In [ ]:
match.head()

In [ ]:
playing11 = pd.read_csv('playing_11.csv')

In [ ]:
playing11.head()

In [ ]:

playing11= playing11.groupby('Players')['Match_ID'].count()

playing11 = pd.DataFrame({'Player':playing11.index, 'Matches':playing11.values})
playing11

In [ ]:
playing11= playing11.sort_values('Matches',ascending=False).reset_index()
playing11= playing11.drop('index',axis=1)
playing11

In [ ]:
data = pd.read_csv('deliveries.csv')
data.head(2)

In [ ]:
# most hatricks
l= ['caught','bowled','lbw','stumped','caught and bowled']
data_bowler = data[data['wicket_type'].isin(l)]
data_bowler['wicket']= 1
data_bowler
df= data_bowler.groupby(['match_id','bowler']).count().reset_index()
df = df[['bowler','wicket']]
#highest wicket
highest_wickets = df.sort_values('wicket',ascending=False).reset_index()
highest_wickets = highest_wickets.drop('index',axis=1)
highest_wickets



In [ ]:
#>=5 wickets
#five_w = 
df= df[df['wicket']>=5]
five_w = df.groupby('bowler')['wicket'].count()
five_w = pd.DataFrame({'bowler':five_w.index,'wicket':five_w.values} )
five_w = five_w.sort_values('wicket',ascending=False).reset_index()
five_w = five_w.drop('index',axis=1)
five_w 

In [ ]:
#most dots bowler
dots_df= data[(data['runs_off_bat']==0) & (data['wides']>0) ]
most_wides = dots_df['bowler'].value_counts().reset_index()
most_wides



In [ ]:
df = data[(data['runs_off_bat']==0)]
df = df['bowler'].value_counts()
dot_wide= pd.DataFrame({'bowler':df.index,'dot_wide':df.values} )
dot_wide

In [ ]:
dot_wide =dot_wide.merge(most_extras,left_on='bowler',right_on='index')
dot_wide

In [ ]:
dot_wide['dots'] = dot_wide['dot_wide']-dot_wide['bowler_y']
dot_wide = dot_wide.sort_values('dots',ascending=False)
dot_wide = dot_wide.drop(columns=['bowler_x','dot_wide','bowler_y'])
dot_wide

In [ ]:
#most extras bowler
dots_df= data[(data['runs_off_bat']==0) & ((data['wides']>0)|(data['legbyes']>0)) ]
most_extras = dots_df['bowler'].value_counts().reset_index()
most_extras


In [ ]:
batsman_run

In [ ]:
import plotly.graph_objects as go
fig = go.Figure([go.Bar(x= batsman_run['striker'].head(10), y=  batsman_run["runs_off_bat"].head(10))])
fig.show()